<a href="https://colab.research.google.com/github/dtoralg/INESDI_Data-Science_ML_IA/blob/main/%5B04%5D%20-%20Modelos%20No%20Supervisados/No_supervisados_Ejercicio_2__pca_wholesale_customers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# No supervisados - Ejercicio 2: pca_wholesale_customers.ipynb

Este notebook es **We do**. Lo trabajamos juntos en clase: verás celdas resueltas que explican los pasos más técnicos y celdas con comentarios `# TODO` que debes completar. Ejecuta las celdas en orden y usa las guías dentro del código para terminar las partes interactivas.

## Objetivos

- Cargar y explorar el dataset *Wholesale Customers*.
- Preparar las features de gasto y estandarizarlas.
- Calcular PCA, analizar la varianza explicada y elegir número de componentes para retener 90% de varianza.
- Reconstruir datos desde componentes y medir el error.
- Comparar el efecto de PCA sobre KMeans (antes/después) y discutir resultados.

In [ ]:
# Librerías y configuración (resuelto)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, silhouette_score

np.random.seed(42)
print('Librerías importadas')

### Carga de datos

Carga el CSV desde la URL si puedes, o sube el archivo al entorno. A continuación tienes un ejemplo comentado y una celda `# TODO` para que hagas la carga.

In [ ]:
# TODO: Carga el dataset Wholesale Customers en un DataFrame `df`.
# Opciones:
# url = 'https://raw.githubusercontent.com/dtoralg/INESDI_Data-Science_ML_IA/refs/heads/main/%5B04%5D%20-%20Modelos%20No%20Supervisados/Wholesale%20customers%20data.csv'
# df = pd.read_csv(url)
# display(df.head())
# print(df.shape)


### Exploración rápida

Revisa tipos, nulos y estadísticas. Observa la estructura y confirma las columnas de gasto.

In [ ]:
# Ejecuta esto si ya has cargado `df` (resuelto)
try:
    display(df.head())
    print('\nInfo:')
    print(df.info())
    print('\n% de nulos por columna:')
    display((df.isnull().mean()*100).round(2))
    print('\nDescripción numérica:')
    display(df.select_dtypes(include=[np.number]).describe().T)
except NameError:
    print('Carga el dataset en la celda de carga antes de ejecutar esta celda')

### Selección de features

Usaremos las columnas de gasto.

In [ ]:
# Resuelto: identifico columnas numéricas y propongo features
# Ajusta la lista si tu CSV tiene nombres distintos
try:
    vars_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
    print('Columnas numéricas detectadas:', vars_numericas)
except NameError:
    print('df no definido. Carga el dataset primero.')

# TODO: Define `features` con las columnas de gasto (p. ej. ['Fresh','Milk','Grocery','Frozen','Detergents_Paper','Delicassen'])
# Crea X = df[features].copy() y muestra X.describe().T


### Escalado

Estandariza las features antes de PCA. El scaler está creado; completa la aplicación sobre `X`.

In [ ]:
# Resuelto: creamos el scaler
scaler = StandardScaler()

# TODO: aplica scaler sobre X y guarda X_scaled como DataFrame con las mismas columnas
# Ejemplo guía (hazlo en clase):
# X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=features)
# display(X_scaled.describe().T)

try:
    X  # comprobación
except NameError:
    print('Define X en la celda de selección de features antes de escalar')

### PCA: cálculo y varianza explicada

Ajusta PCA y muestra la varianza explicada. Elige el `target_var` (ej. 0.90) en la celda siguiente y calcula cuántas componentes son necesarias.

In [ ]:
# Resuelto: cálculo inicial de PCA (requiere X_scaled)
try:
    pca_full = PCA()
    X_pca_full = pca_full.fit_transform(X_scaled)
    explained = pca_full.explained_variance_ratio_
    cumulative = np.cumsum(explained)
    explained_df = pd.DataFrame({'pc': np.arange(1, len(explained)+1), 'explained_variance': explained, 'cumulative': cumulative})
    explained_df.index = explained_df['pc']
    display(explained_df[['explained_variance','cumulative']])

    plt.figure(figsize=(8,4))
    plt.plot(explained_df['pc'], explained_df['explained_variance'], marker='o', label='Individual')
    plt.plot(explained_df['pc'], explained_df['cumulative'], marker='o', label='Cumulative')
    plt.xlabel('Componente principal')
    plt.ylabel('Varianza explicada')
    plt.xticks(explained_df['pc'])
    plt.legend()
    plt.title('Scree plot y varianza acumulada')
    plt.grid(True)
    plt.show()

    # TODO: en clase elige target_var (ej. 0.90) y calcula n_components_90 = np.argmax(cumulative >= target_var) + 1
    # Luego crea pca90 = PCA(n_components=n_components_90) y X_pca_90 = pca90.fit_transform(X_scaled)

except Exception as e:
    print('Asegúrate de tener X_scaled definido. Detalle:', e)

### Reconstrucción y error (guía)

Vamos a medir la pérdida de información al reconstruir los datos desde un número reducido de componentes. Completa el bucle para k

In [ ]:
# Ejemplo resuelto parcial y guía para completar
try:
    from sklearn.metrics import mean_squared_error
    # Ejemplo: reconstrucción y MSE con k=2 (modifica en clase)
    k_example = min(2, X_scaled.shape[1])
    pca_k = PCA(n_components=k_example)
    Xk = pca_k.fit_transform(X_scaled)
    Xk_rec = pca_k.inverse_transform(Xk)
    mse_example = mean_squared_error(X_scaled, Xk_rec)
    print(f'MSE reconstrucción con k={k_example}:', mse_example)

    # TODO: recorre k desde 1 hasta X_scaled.shape[1], calcula MSE para cada k y guarda resultados en mse_df. Dibuja MSE vs k.

except NameError:
    print('Asegúrate de definir X_scaled antes de ejecutar esta celda')

### PCA antes/después de clustering

Queremos comparar cómo cambia la inercia de KMeans al aplicar PCA antes del clustering. Completa la comparación y revisa cuándo tiene sentido usar PCA como preprocesado.

In [ ]:
# Guía para la comparativa (resuelto parcialmente)
# TODO: completa estos pasos en clase:
# - define k_chosen (por ejemplo 4 o el valor que acordemos)
# - km_orig = KMeans(n_clusters=k_chosen, random_state=42, n_init=10).fit(X_scaled); inertia_orig = km_orig.inertia_
# - km_pca = KMeans(n_clusters=k_chosen, random_state=42, n_init=10).fit(X_pca_90) (si has calculado X_pca_90)
# - imprime inertia_orig e inertia_pca y comenta la diferencia

### Interpretación y entrega

Redacta en la última celda un resumen (3–5 líneas) con el número de componentes que decidiste retener, el error de reconstrucción observado y si recomendarías usar PCA antes de clustering en este caso.

In [ ]:
# Escribe tu resumen final aquí (sustituye el texto entre comillas):
resumen = '''

'''
print(resumen)